# Machine Learning on MCU - Ex3.2

This exercise will introduce you to Keras, one of the most popular frameworks used in Deep Learning.

You will train a simple multi-layer perceptron to predict Human Activity from Smartphone Accelerometer and Gyroscope Data.

We use a dataset of 3-axial accelerometer signals from an academic experiment on the UC Irvine Machine Learning Repository.

The dataset is downloaded in the code snippet below and you can you can also find the description of the dataset [here](https://archive.ics.uci.edu/dataset/341/smartphone+based+recognition+of+human+activities+and+postural+transitions) .

# Load the dataset

In [ ]:
# If required, download the dataset
import requests
import os.path
import zipfile
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

random.seed(7)

if (not os.path.isdir('./HAPT Data Set')):
    open('./HAPT Data Set.zip', 'wb').write(requests.get(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip", 
        allow_redirects=True).content)
    zipfile.ZipFile('./HAPT Data Set.zip', 'r').extractall('./HAPT Data Set')

Load the accelerometer and gyroscope data.
We read the feature names from features.txt and the activity labels from activity_labels.txt

In [ ]:
with open('./HAPT Data Set/features.txt') as f:
    features = f.read().split()

print('There are {} features.'.format(len(features)))
    
with open('./HAPT Data Set/activity_labels.txt') as f:
    activity_labels = f.readlines()

activity_df = [x.split() for x in activity_labels]
print('There are {} activities.'.format(len(activity_df)))
pd.DataFrame(activity_df, columns = ['Activity_id', 'Activity_label'])

The data are pre-split into training and test sets. Let's load the features x and the labels y, and have a look at a few features.

In [ ]:
X_train = pd.read_table('./HAPT Data Set/Train/X_train.txt',
             header = None, sep = " ")
X_train.iloc[:10, :10].head()

In [ ]:
y_train = pd.read_table('./HAPT Data Set/Train/y_train.txt',
             header = None, sep = " ", names = ['Activity_id'])
y_train.head()

In [ ]:
X_test = pd.read_table('./HAPT Data Set/Test/X_test.txt',
             header = None, sep = " ")
y_test = pd.read_table('./HAPT Data Set/Test/y_test.txt',
             header = None, sep = " ", names = ['Activity_id'])

## Predict Human Activity

In [ ]:
# Import the necessary packages


# Note: use Tensor Flow backend for Keras as suggested on keras.io:
# "At this time, we recommend that Keras users who use multi-backend Keras with the TensorFlow backend switch to tf.keras in TensorFlow 2.0. tf.keras is better maintained and has better integration with TensorFlow features (eager execution, distribution support and other)."
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models, regularizers

In [ ]:
# Load the accelerometer and gyroscope data.
X_train_full = X_train.values
X_test_data = X_test.values
y_train_full = y_train.values.flatten()
y_test_data = y_test.values.flatten()

print("Training samples: {}, Test samples: {}".format(X_train_full.shape[0], X_test_data.shape[0]))
print("Number of features: {}".format(X_train_full.shape[1]))

In [ ]:
# Split the dataset into training, validation, and testing sets.
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=7)

print("Training: {}, Validation: {}, Test: {}".format(
    X_tr.shape[0], X_val.shape[0], X_test_data.shape[0]))

In [ ]:
# Shift the class labels by -1 so they start from 0
y_tr = y_tr - 1
y_val = y_val - 1
y_test_shifted = y_test_data - 1

num_classes = len(np.unique(y_train_full))
num_features = X_tr.shape[1]
print("Number of classes: {}".format(num_classes))
print("Number of features: {}".format(num_features))

In [ ]:
# Define batchsize and number of epochs
batch_size = 32
epochs = 50

# Declare the sequential model and design your multi-layer perceptron
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_features,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile your model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train your model
history = model.fit(X_tr, y_tr,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val),
                    verbose=1)

In [ ]:
# Model summary
model.summary()

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(history.history['loss'], label='Train Loss')
ax1.plot(history.history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.set_title('Loss')

ax2.plot(history.history['accuracy'], label='Train Accuracy')
ax2.plot(history.history['val_accuracy'], label='Val Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.set_title('Accuracy')
plt.show()

In [ ]:
# Evaluate your model on the test set
test_loss, test_acc = model.evaluate(X_test_data, y_test_shifted, verbose=1)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

In [ ]:
# Save your model once you are satisfied with it
model.save('mlp_har_model.keras')
print("Model saved to mlp_har_model.keras")

In [ ]:
# Load your saved model and test on the test data.
loaded_model = keras.models.load_model('mlp_har_model.keras')

test_loss, test_acc = loaded_model.evaluate(X_test_data, y_test_shifted, verbose=1)
print("Loaded model test accuracy: {:.2f}%".format(test_acc * 100))